In [1]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [2]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

/home/svcl-oowl/brandon/research/sil_consistent_at_inference


In [95]:
import pprint
import glob
import os

import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from tqdm import tqdm


In [98]:
#IMAGE_DATASET_PATH = 'data/pix3d_transparent_224/chair'
IMAGE_DATASET_PATH = 'data/shapenet_chair_images_transparent'

new_image_size = 224

In [99]:
# given an RGB image of an object on a white background, will segment the white background out and return an RGBA image
# this is done by binary contouring and some morphological operations to clean up noise

def get_mask(rgb_img):
    gray_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
    ret,mask = cv2.threshold(gray_img,245,255,cv2.THRESH_BINARY)
    kernel = np.ones((3,3),np.uint8)
    mask = cv2.dilate(mask,kernel,iterations = 1)
    mask = cv2.erode(mask,kernel,iterations = 1)
    return mask

def remove_white_background(rgb_img):
    mask = get_mask(rgb_img)
    rgba_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2RGBA)
    rgba_img[:, :, 3] = cv2.bitwise_not(mask)
    return rgba_img

In [ ]:
# convert images to make all white pixels transparent
# based off https://stackoverflow.com/questions/765736/using-pil-to-make-all-white-pixels-transparent
#TODO: check bgr
i = 0
for root, dirs, files in os.walk(os.path.join(IMAGE_DATASET_PATH)):
    for f in tqdm(files):
        image_path = os.path.join(root,f)
        print(image_path)
        rgb_img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        rgba_img = remove_white_background(rgb_img)
        
        rgba_img_name = f.split('.')
        rgba_img_name[-1] = 'png'
        rgba_img_name = '.'.join(rgba_img_name)
        cv2.imwrite(os.path.join(root,rgba_img_name), rgba_img)
        cv2.imwrite(os.path.join(root,rgba_img_name), cv2.cvtColor(rgba_img, cv2.COLOR_RGBA2BGRA))
        
        os.remove(image_path)


In [ ]:
# resize images
for root, dirs, files in os.walk(os.path.join(IMAGE_DATASET_PATH)):
    for f in files:
        image_path = os.path.join(root,f)
        image = Image.open(image_path).resize((new_image_size,new_image_size), resample = PIL.Image.BILINEAR)
        image.save(image_path)